# Envision 2022

## Basic Setup

Importing and Loading the URDF/SDF

In [ ]:
import pybullet as p

physicsClientID = p.connect(p.GUI)

import pybullet_data

p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [ ]:
def loadFiles():
    plane = p.loadURDF('plane.urdf')

    robot_StartPos = [0,0,0]
    robot_StartOrientation = p.getQuaternionFromEuler([0,0,0])

    robot = p.loadSDF("/kuka_iiwa/kuka_with_gripper2.sdf")[0]
    p.resetBasePositionAndOrientation(robot, robot_StartPos, robot_StartOrientation)

    numOfJoints = p.getNumJoints(robot)

    print(f"Plane = {plane} \nStart Position = {robot_StartPos} \nStart Orientation = {robot_StartOrientation} \nRobot = {robot} \nNumber of joints = {numOfJoints}")

    # print("Joint Details :")
    # for i in range(numOfJoints):
    #     print(p.getJointInfo(robot, i))
        
    return plane, robot, robot_StartPos, robot_StartOrientation

## Setting Up Camera

In [ ]:
def getImage(robot) :
    
    # The Camera will be placed on the Link 7 and will always be looking downwards
    
    # The Position and orientation of the Link 7 will be used to determine the camera position and orientation
    cameraPosition = p.getLinkState(robot, 7)[0]
    cameraOrientation = p.getEulerFromQuaternion(p.getLinkState(robot, 7)[1])

    print(f"\nCamera Position: {cameraPosition} \nCamera Orientation: {cameraOrientation}")
    
    gainX = gainY = gainZ = 2
    
    if cameraOrientation[0] < 0:
        gainX = -2
        
    if cameraOrientation[1] < 0:
        gainY = -2
        
    if cameraOrientation[2] < 0:
        gainZ = -2

    # The viewMatrix The camera view matrix is a complicated 4x4 matrix, but in simplest terms it describes where the camera is and in what direction it is pointing. There are some useful helper functions for creating this matrix by specifying position and rotation more directly. The function computeViewMatrix can create this matrix in exchange for three vectors.
    viewMatrix = p.computeViewMatrix(
        # cameraEyePosition describes the physical location of the camera in x, y, and z coordinates
        cameraEyePosition = cameraPosition,
        # cameraTargetPosition describes the point that we wish the camera to face : Negative Z direction, that is downwards
        cameraTargetPosition = [cameraOrientation[0] * 100 , cameraOrientation[1] * 100, cameraOrientation[2] * -100],
        # cameraUpVector describes the orientation of the camera
        cameraUpVector = [cameraOrientation[0] * 1 , cameraOrientation[1] * 1, cameraOrientation[2] * -1])

    # The projection matrix, much like the view matrix, can be created using a few helper functions. In this case, the computeProjectionMatrixFOV function describes our camera’s intrinsic properties in the simplest and most pertinent ways to our use case.
    projectionMatrix = p.computeProjectionMatrixFOV(
        # Field of View
        fov = 45.0,
        # Aspect ratio
        aspect = 1.0,
        # The minimum distance the camera will render objects
        nearVal = 0.1,
        # The maximum distance the camera will render objects
        farVal = 10.1)

    # This function returns three image buffers: rgbImg, depthImg, and segImg. rgbImg is a uint8 image with red, green, blue, and alpha channels of the camera’s visuals. depthImg is a floating-point grayscale image that describes the distance of individual rendered pixels from the camera. It can be used to model the field-of-view of a real-world depth sensor. Lastly, segImg is a segmentation mask of the image where pixels each contain unique integers with object IDs. These are invaluable for training segmentation algorithms for robotic agents, such as a robotic arm that needs to identify objects to sort into respective bins or for a driverless car that wants to identify pedestrians, street signs, and roads.
    width, height, rgbImg, depthImg, segImg = p.getCameraImage(
        # Width of the image in pixels
        width = 224, 
        # Height of the image in pixels
        height = 224,
        # view matrix as the one deifined above
        viewMatrix = viewMatrix,
        # projectionMatrix as the one deifined above
        projectionMatrix = projectionMatrix)
    
    return width, height, rgbImg, depthImg, segImg

## Function Calls

In [ ]:
plane, robot, robot_StartPos, robot_StartOrientation = loadFiles()

In [ ]:
width, height, rgbImg, depthImg, segImg = getImage(robot)

## Simulation parameters

In [ ]:
p.setRealTimeSimulation(1)

In [ ]:
p.setRealTimeSimulation(0)
p.resetSimulation()

## Rough

In [ ]:
print("Joint Details :")
for i in range(14):
    # print(p.getJointInfo(robot, i)[0:3], p.getJointInfo(robot, i)[8:10])
    print(p.getJointInfo(robot, i))

In [ ]:
p.setJointMotorControlArray(robot, [8, 11], p.POSITION_CONTROL, targetPositions=[0.0,0], forces = [0.0001]*2)

In [ ]:
p.setJointMotorControl2(robot, 1, p.VELOCITY_CONTROL, targetVelocity = 0.1)

In [ ]:
p.setJointMotorControl2(robot, 1, p.VELOCITY_CONTROL, targetVelocity = 0.0)

In [ ]:
print(p.getLinkState(robot, 8)[0], "\n", p.getEulerFromQuaternion(p.getLinkState(robot,8)[1]))